In [2]:
!pwd

/home/viskuit/Desktop/Proyecto_Leishmania/RepeatModeler/RepeatMask_With_SIDER2-Chr1-data/Blastdbcmd


In [3]:
import csv
import subprocess
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

In [4]:
!gawk 'BEGIN {RS="\n"}{print $0;}' L_infantum_ALL_36Chr.fasta.out | gawk '{if ($7 - $6 > 449) print $5,$6,$7,$9,$11;}' > PRUEBA1.txt

In [6]:
!gawk 'BEGIN {RS="\n"}{print $5;}' PRUEBA1.txt | sort | uniq -c

     10 Simple_repeat
    446 Unspecified


In [7]:
!grep Unspecified PRUEBA1.txt > PRUEBA2.txt

In [4]:
genome_fasta = "./L_infantum_ALL_36Chr.fasta"
new_data = []
with open("./PRUEBA2.txt", "r") as main_file:
    reader = csv.reader(main_file, delimiter=" ")
    for row in reader:
        if "+" in row[3]:
            seq = subprocess.check_output("blastdbcmd -db " + genome_fasta + " -entry "
                                          + row[0] + " -range " + row[1] + "-" + row[2]
                                          + " -strand plus -outfmt %s",
                                          shell=True,
                                          universal_newlines=True)  # Very important to use "subprocess.check_output" so we can get the output
            seq = seq.strip()  # Remove EoL characters
            new_row = [row[0], "plus", str(len(seq)), row[1], row[2], seq]
            new_data.append(new_row)
        elif "C" in row[3]:
            seq = subprocess.check_output("blastdbcmd -db " + genome_fasta + " -entry "
                                          + row[0] + " -range " + row[1] + "-" + row[2]
                                          + " -strand minus -outfmt %s",
                                          shell=True,
                                          universal_newlines=True)  # Very important to use "subprocess.check_output" so we can get the output
            seq = seq.strip()  # Remove EoL characters
            new_row = [row[0], "minus", str(len(seq)), row[1], row[2], seq]
            new_data.append(new_row)

In [5]:
len(new_data)

446

In [6]:
new_data[0]

['LinJ.01',
 'plus',
 '660',
 '24100',
 '24759',
 'GGCGGGGGAGGCGGGGGGCACGCACCTCCATGCGTGGCATCCCAGGGTCCAGCGCCCCCCCCCTCCACCCCCGCTCTCTCTGTGTACGGAAGCCCGGCAGCCCCCTCACCCTCTATCCCTGCCAACGCCGAACCACTTCTGGTGCTGACAGGGCTGGGCCCTTACGGCGTTGGGCGAGGTCGGCGCGACGTAGCGCCACGGATGCCGGCGGCCATGTCGTGCATGGCGCTGCGTCCGAGCCACCCGCGACAGTGAGCACAGGCTTGCACGGTCCATGCGATGGGCGGAGCGTGTCAGCGCGGCTCGAGCGTGTCGCAGCCGGCCCCTCACACTGGCCCACGGCGAGGGGTGCGGGGGCCTGAGTGTCACCGCGACGGGGAGACGCACCCAGGGAGGAGGTGGGGGGAGTGGGGACCGGCATGATGGAGGGGCGGCTGTGTGGCGATGTGTGTGTATGTGGGTGTCGTCGCGTTTGAGGCAGGAGCCGTGCTGCGACGACCGAGCCGGCGCACTGCTGCAGCGCGCGTGTGTCTTGCGCTGCTTCGCACCAGGCGATGAGAGTGGGGTGGGGTGCCTGCAGCACTCGGCGGCGGGGGGGTGCAGAGCGGCCTCCACCTCGCAGTGTGCGGCAGCGAAAACGGACACGCGGAAGAGCAAACA']

In [7]:
# Now let's transform it to a fasta file
fasta_data = []
for sequence in new_data:
    rec = SeqRecord(
        Seq(sequence[5]),
        id=sequence[0] + " " + sequence[1] + " length:" + sequence[2] + " " + sequence[3] + "-" + sequence[4],
        description="Leishmania infantum"
    )

    fasta_data.append(rec)

In [8]:
SeqIO.write(fasta_data, "./PRUEBA2.fasta", "fasta")

446